# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy.stats import linregress

#import API key
from api_keys import weather_api_key

#import citipy to determine city based on latitude and longitude
from citipy import citipy

#import geopy to find country based on latitude and longitude
from geopy.geocoders import Nominatim

#output to File (CSV)
output_data_file = "output_data/cities.csv"

#range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
#list for holding lat/lng and cities
lat_lngs = []
cities = []

#create a set of random lat and lng combinations, zip into one list variable
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)


#identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #if the city is not already in our cities list, add it
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

655

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:

url = 'http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=' + weather_api_key + '&q=' 

cityData = []
cityDataStats = []
set_count = 1 #group
record = 1 #record within group

print(f'Beginning Data Retrieval')
print(f'----------------------------')

for i, city in enumerate(cities):
    if(i % 50 == 0 and i >= 50):
        set_count += 1 #make new group
        record = 0 #reset records in the group 
        
#output record and group being processed
    print(f'processing record {record} of set {set_count} | {city}')       
     
    record += 1
    
    cityurl = url + city
    try:
        cityWeather = requests.get(cityurl).json()
        lat = cityWeather['coord']['lat']
        lon = cityWeather['coord']['lon']
        maxTemp = cityWeather['main']['temp_max']
        humidity = cityWeather['main']['humidity']
        cloudiness = cityWeather['clouds']['all']
        windSpeed = cityWeather['wind']['speed']
        date = cityWeather['dt']
        timestamp = datetime.datetime.fromtimestamp(date)
        dateConvert = timestamp.strftime('%m-%d-%Y At %H:%M:%S')
        geolocator = Nominatim(user_agent="python_api_project")
        location = geolocator.reverse(str(lat)+","+str(lon))
        address = location.raw['address']
        country_code = address.get('country_code').upper()
        
        cityData.append({'City':city, 'Lat':round(lat, 2), 'Lng':round(lon, 2), 'Max Temp': str(maxTemp) + ' F', 
                         'Humidity':str(humidity) + '%', 'Cloudiness':str(cloudiness) + '%', 'Wind Speed':str(windSpeed) + ' MPH',
                        'Country': country_code, 'Date':dateConvert})
        cityDataStats.append({'Lat':lat, 'Lng':lon, 'Max Temp': maxTemp, 
                         'Humidity':humidity, 'Cloudiness':cloudiness, 'Wind Speed':windSpeed,
                        'Date':date})

        
        

    except:
        print(f'{city} not found. Skipping...')
     
    if (i == 19):
        break

Beginning Data Retrieval
----------------------------
processing record 1 of set 1 | barrow
processing record 2 of set 1 | kapaa
processing record 3 of set 1 | wa
wa not found. Skipping...
processing record 4 of set 1 | carnarvon
processing record 5 of set 1 | ushuaia
processing record 6 of set 1 | hermanus
processing record 7 of set 1 | zhanaozen
processing record 8 of set 1 | fortuna
processing record 9 of set 1 | kilcoole
processing record 10 of set 1 | sumbawa
sumbawa not found. Skipping...
processing record 11 of set 1 | punta arenas
processing record 12 of set 1 | saint-augustin
processing record 13 of set 1 | rikitea
processing record 14 of set 1 | ilulissat
processing record 15 of set 1 | bredasdorp
processing record 16 of set 1 | robertsport
processing record 17 of set 1 | lebu
processing record 18 of set 1 | sola
processing record 19 of set 1 | hithadhoo
processing record 20 of set 1 | mataura


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [4]:
cityData_pd = pd.DataFrame(cityData)
cityData_pd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.29,-156.79,10.4 F,85%,1%,4.61 MPH,US,04-23-2021 At 09:02:43
1,kapaa,22.08,-159.32,73.4 F,78%,75%,10.36 MPH,US,04-23-2021 At 09:02:48
2,carnarvon,-24.87,113.63,69.8 F,83%,1%,10.36 MPH,AU,04-23-2021 At 09:03:55
3,ushuaia,-54.80,-68.30,42.8 F,87%,90%,2.26 MPH,AR,04-23-2021 At 08:57:44
4,hermanus,-34.42,19.23,63 F,76%,70%,1.99 MPH,ZA,04-23-2021 At 09:00:20


In [5]:
cityDataStats_pd = pd.DataFrame(cityDataStats)
cityDataStats_pd.describe(datetime_is_numeric=True)

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,1.800000e+01
mean,1.910122,-16.671056,58.336111,72.166667,55.611111,9.216111,1.619194e+09
std,43.661488,102.421009,19.192542,15.313393,38.646420,6.821671,1.407769e+02
min,-54.800000,-159.319000,10.400000,36.000000,1.000000,1.990000,1.619193e+09
25%,-34.503825,-72.966675,45.350000,65.750000,22.750000,3.580000,1.619194e+09
50%,-7.241650,-31.234300,59.120000,72.500000,61.000000,5.840000,1.619194e+09
75%,42.624550,44.610075,72.500000,82.500000,93.750000,14.340000,1.619194e+09
max,71.290600,168.864300,82.870000,96.000000,100.000000,21.050000,1.619194e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
#get the indices of cities that have humidity over 100%.
cityDataClean_pd = cityData_pd
for i, city in enumerate(cities):
    url = 'http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=' + weather_api_key + '&q='
    cityurl = url + city
    try:
        cityWeather = requests.get(cityurl).json()
        if(int(cityWeather['main']['humidity']) > 100):
            cityDataClean.drop(i, axis=0)
    except:
        print('Humidity data not retrievable. Skipping...')
cityDataClean_pd.head(10)
        

Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity data not retrievable. Skipping...
Humidity da

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.29,-156.79,10.4 F,85%,1%,4.61 MPH,US,04-23-2021 At 09:02:43
1,kapaa,22.08,-159.32,73.4 F,78%,75%,10.36 MPH,US,04-23-2021 At 09:02:48
2,carnarvon,-24.87,113.63,69.8 F,83%,1%,10.36 MPH,AU,04-23-2021 At 09:03:55
3,ushuaia,-54.80,-68.30,42.8 F,87%,90%,2.26 MPH,AR,04-23-2021 At 08:57:44
4,hermanus,-34.42,19.23,63 F,76%,70%,1.99 MPH,ZA,04-23-2021 At 09:00:20
5,zhanaozen,43.30,52.80,59.38 F,55%,22%,21.05 MPH,KZ,04-23-2021 At 09:05:22
6,fortuna,40.60,-124.16,55.99 F,81%,1%,3.15 MPH,US,04-23-2021 At 09:02:00
7,kilcoole,53.10,-6.07,57 F,36%,100%,3 MPH,IE,04-23-2021 At 09:05:23
8,punta arenas,-53.15,-70.92,46.4 F,93%,90%,3.44 MPH,CL,04-23-2021 At 08:59:53
9,saint-augustin,51.23,-58.65,37.08 F,65%,100%,17.36 MPH,CA,04-23-2021 At 09:01:34


In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression